# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'ht

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/meta-llama/Llama-4-Scout-17B-16E-Instruct',
 '/agentica-org/DeepCoder-14B-Preview',
 '/HiDream-ai/HiDream-I1-Full',
 '/reducto/RolmOCR',
 '/moonshotai/Kimi-VL-A3B-Thinking',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/jamesliu1217/EasyControl_Ghibli',
 '/spaces/Stable-X/Hi3DGen',
 '/spaces/VAST-AI/TripoSG',
 '/spaces/Efficient-Large-Model/SanaSprint',
 '/spaces',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/open-thoughts/OpenThoughts2-1M',
 '/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset',
 '/datasets/agentica-org/DeepCoder-Preview-Dataset',
 '/datasets/OmniSVG/MMSVG-Illustration',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/In

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on m

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmeta-llama/Llama-4-Scout-17B-16E-Instruct\nUpdated\n4 days ago\n•\n445k\n•\n746\nagentica-org/DeepCoder-14B-Preview\nUpdated\n3 days ago\n•\n6.28k\n•\n395\nHiDream-ai/HiDream-I1-Full\nUpdated\n3 days ago\n•\n4.82k\n•\n306\nreducto/RolmOCR\nUpdated\n10 days ago\n•\n5.44k\n•\n332\nmoonshotai/Kimi-VL-A3B-Thinking\nUpdated\nabout 15 hours ago\n•\n3.19k\n•\n218\nBrowse 1M+ models\nSpaces\nRunning\n

In [22]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    return response.choices[0].message.content

In [23]:
#create_brochure("HuggingFace", "https://huggingface.co")
eng_brochure = create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}, {'type': 'company social media', 'url': 'https://twitter.com/huggingface'}, {'type': 'company social media', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


In [24]:
eng_brocher = create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}]}


In [28]:
def get_translation_brochure_user_prompt(company_name, eng_brocher, language):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here is the brocher in english for the company; translate this brochure in {language} in markdown.\n"
    user_prompt += eng_brocher
    user_prompt = user_prompt[:10_000] # Truncate if more than 10,000 characters
    return user_prompt

In [29]:
system_prompt_translation = "You are an assistant that translates the contents of company  brocher in English into different languages \
and creates a short brochure about the company for prospective customers, investors and recruits in different regions and nationalities. Respond in markdown.\
Make sure that links remain still valid."

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [34]:
create_brochure_translation("HuggingFace", eng_brocher, "Urdu") 

# Hugging Face بروشر

---

## ہمارے بارے میں
**Hugging Face** ایک متحرک AI کمیونٹی ہے جو مشین لرننگ کے مستقبل کی تشکیل کے لیے وقف ہے۔ ہم ایک تعاون پر مبنی پلیٹ فارم فراہم کرتے ہیں جہاں افراد اور تنظیمیں مل کر ماڈلز، ڈیٹا سیٹس، اور ایپلی کیشنز کا جائزہ لیتے ہیں، جدت طرازی کرتے ہیں، اور انہیں بانٹتے ہیں۔ ہمارا مشن مشین لرننگ کی فن اور سائنس کو تیز کرنا ہے، اسے سب کے لئے قابل رسائی بنانا ہے۔

---

## ہماری پیشکشیں
### ماڈلز کی تلاش کریں
**1M+ ماڈلز** کے ساتھ، صارفین جدید AI ٹیکنالوجی کو براؤز اور استعمال کر سکتے ہیں۔ ہمارا پلیٹ فارم متروک ماڈلز کی میزبانی کرتا ہے جیسے:
- **meta-llama/Llama-4-Scout-17B-16E-Instruct**
- **agentica-org/DeepCoder-14B-Preview**
- **HiDream-ai/HiDream-I1-Full**

### ڈیٹا سیٹس
ہم **250k+ ڈیٹا سیٹس** تک رسائی فراہم کرتے ہیں، جس سے محققین اور ڈویلپرز اپنے پروجیکٹس کو قیمتی، اعلیٰ معیار کے ڈیٹا سے بھرپور کر سکتے ہیں۔

### ایپلی کیشن کی جگہیں
ہمارے **4k ایپلی کیشنز** کے کمیونٹی کا حصہ بنیں جو امیج جنریشن سے ماڈل ترجمے تک مختلف شعبوں میں جدت کی سہولت فراہم کرتی ہیں۔

---

## ہمارے صارفین
Hugging Face پر **50,000 سے زیادہ تنظیموں** کا اعتماد ہے، جن میں شامل ہیں:
- **Meta**
- **Amazon**
- **Google**
- **Microsoft**
یہ کمپنیاں ہمارے جدید ماڈلز اور ڈیٹا سیٹس کا استعمال کرکے اپنی AI پہلوں میں جدت کو فروغ دیتی ہیں۔

---

## کمپنی کا ثقافت
Hugging Face میں ہماری ثقافت ایک تعاون، شفافیت، اور اوپن سورس کی وابستگی پر مبنی ہے۔ ہم کمیونٹی کے ساتھ مل کر تعمیر کرنے پر یقین رکھتے ہیں، اکثر صارفین کی شراکت داری اور فیڈبیک کا استعمال کرتے ہوئے اپنے ٹولز اور سروسز کی تشکیل کرتے ہیں۔ ہماری ٹیم ایک معاون ماحول میں ترقی کرتی ہے جو جدت کو فروغ دیتی ہے اور ذاتی نشونما کو بڑھاتی ہے۔

### کمیونٹی کی شمولیت
ہم فورمز، مشترکہ پروجیکٹس، اور مباحثے کی میزبانی کرتے ہیں جو اراکین کو بصیرتیں شیئر کرنے، مسائل کا حل نکالنے، اور کامیابیوں کا جشن منانے کے مواقع فراہم کرتے ہیں۔ ٹولز کے اوپن سورس کی وابستگی ہماری AI تک رسائی کو جمہوری بنانے کے یقین کو مضبوط کرتی ہے۔

---

## Hugging Face میں کیریئر
آئیں ہم مشین لرننگ کا انقلاب لائیں! ہم ایسے پرجوش افراد کی تلاش میں ہیں جو ہماری مشن میں شرکت کے لیے تیار ہیں۔ کھلی عہدے مختلف شعبوں میں انجینئرنگ، تحقیق، پروڈکٹ مینجمنٹ، اور کمیونٹی کی شمولیت شامل ہیں۔ دستیاب مواقع کے لیے ہماری کیریئرز کے صفحے کو دیکھیں اور AI کے مستقبل کو تشکیل دینے کے اس دلچسپ سفر کا حصہ بنیں!

---

## ہم سے جڑیں
چاہے آپ اپنی AI ایپلی کیشنز کے لیے ہمارے پلیٹ فارم کا فائدہ اٹھانے کے لیے دیکھ رہے ہوں، ہمارے ساتھ شراکت کرنا چاہتے ہوں، یا ہماری متحرک ٹیم میں شامل ہونا چاہتے ہوں، ہم آپ کو ہم سے جڑنے کی دعوت دیتے ہیں۔

- [ہمارے ویب سائٹ پر وزٹ کریں](https://huggingface.co/)
- ہمیں [Twitter](https://twitter.com/huggingface) | [LinkedIn](https://linkedin.com/company/huggingface) | [GitHub](https://github.com/huggingface) پر فالو کریں

آئیں، مل کر AI کے مستقبل کی تعمیر کریں!

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [35]:
def create_brochure_translation(company_name, eng_brocher, language):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt_translation},
            {"role": "user", "content": get_translation_brochure_user_prompt(company_name, eng_brocher, language)}
          ],
    )
    display(Markdown(response.choices[0].message.content))

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face: Transforming AI Together

## About Us
Hugging Face is an innovative AI community dedicated to building the future of machine learning (ML). We provide a platform where users can create, collaborate, and share machine learning models, datasets, and applications. With over **1 million models** and **250k datasets**, our hub promotes exploration, transparency, and collaboration across the global AI landscape.

## Our Offerings
- **Collaboration**: Hugging Face acts as a comprehensive collaboration platform for ML professionals, enabling the hosting and sharing of unlimited public models and datasets.
- **Advanced Solutions**: We offer paid compute and enterprise solutions, facilitating the development of AI with sophisticated security, access controls, and dedicated support for teams.
- **Open Source**: Our commitment to the open-source community means we continually develop powerful tools and libraries, including Transformers, Diffusers, and Tokenizers, that empower researchers and developers alike.

## Who We Serve
Hugging Face partners with over **50,000 organizations**, including major players like Meta, Amazon, Google, and Microsoft. Whether you’re a developer, researcher, or enterprise, our platform provides the resources and support needed to thrive in the AI space.

## Company Culture
Our culture emphasizes collaboration, inclusion, and innovation. We believe in the power of community, and our team is dedicated to fostering an environment that supports diversity and encourages creative solutions. Each member of Hugging Face plays a key role in advancing ML and contributing to the open-source ecosystem.

## Careers at Hugging Face
We are always on the lookout for passionate individuals to join our team! Whether you are a seasoned professional or just starting in your career, we uphold a dynamic work environment that values creativity and initiative. Explore our current job openings and help us build the future of AI.

## Join Us!
Be part of the AI revolution! Collaborate with us on cutting-edge projects, build your ML portfolio, and contribute to a vibrant community. Sign up today and start exploring the endless possibilities that Hugging Face has to offer.

### Contact Us
For more information, visit our website at [huggingface.co](https://huggingface.co) or follow us on social platforms like GitHub, Twitter, and LinkedIn.

---

Together, let's shape the future of AI!

In [21]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Company Brochure

## Welcome to Hugging Face
**The AI community building the future.**  
At Hugging Face, we are dedicated to creating a collaborative environment for the machine learning community to explore, create, and innovate. Our platform serves as a comprehensive resource for models, datasets, and applications, aiming to accelerate the pace of AI development.

---

## What We Do
- **Models**: Browse through over **1 million models**, including the latest trending innovations in AI. Our diverse offerings cater to various use cases across text, image, video, audio, and even 3D modalities.
  
- **Datasets**: Access and share a rich library of **250,000+ datasets** designed to enhance your machine learning tasks.
  
- **Spaces**: Create and collaborate on thousands of applications directly on our platform, allowing users to innovate and deploy without limitations.

- **Enterprise Solutions**: We offer tailored solutions for organizations, including enterprise-grade security, dedicated support, and affordable pricing starting at **$20/user/month**.

---

## Our Customers
More than **50,000 organizations** trust Hugging Face, including industry giants such as:

- **AI at Meta**
- **Google**
- **Microsoft**
- **Amazon**
  
Our community-driven approach ensures that we continually grow and improve, catering to both large enterprises and individual developers alike.

---

## Company Culture
At Hugging Face, we champion a collaborative and inclusive company culture. Our open-source mission drives transparency and encourages community participation in the development of machine learning tools. We believe in building the future of AI together, fostering innovation while maintaining a commitment to ethical AI practices.

---

## Careers at Hugging Face
Would you like to be part of a dynamic and passionate team that is shaping the future of AI? We are always on the lookout for talented individuals to join our journey. You can explore current openings in various domains to find a role that matches your skills and ambitions. Join us in making machine learning accessible to everyone!

---

## Get Involved
Whether you are a developer, data scientist, or just enthusiastic about AI, Hugging Face offers a unique opportunity to collaborate and innovate. Sign up today to start exploring our models and datasets, and maximize your impact in the AI community.

[**Join us at Hugging Face!**](https://huggingface.co)  

--- 

### Connect with Us
- **GitHub**
- **Twitter**
- **LinkedIn**
- **Discord**

**Hugging Face – Your partner in AI innovation!**

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>